# Yu's capstone project
## Introduction

###   As the top 2 populated and famous regions in Canada, Greater Vancouver Area and Toronto are two symbol cities of east and west coast. While the differences among geographical locations and development patterns lead these two places to the separate ways. So in this capstone project, I'll try to find the similarities and dissimilarities between them based on the top most common venues, which can be used to organize the neighborhoods into several clusters and then easily for comparison. Moreover, I believe the similarities and dissimilarities will display what is abundant and what is lacking in these two regions because the proportion of venues' types can be calculated from the data, and that is exactly what the current and future investors and developers want to learn about. Hence, it somehow tells the investors what is the situation and developing stage of the cities now as well as indicating the next steps or goals they could attempt to make. 

### - For both Vancouver and Toronto data sets, I'll load them from wikipedia and make some transformation so that the outcome and the format will be properly visualized and defined.
### - After the adjustment, I'll set the radius and use the foursquare api so that a detailed information can be obtained, which already includes the venues. Then the categories and locations will be shown to us.
### - In the next couple of steps, I'll calculate the frequency of each category in each neighborhood from Vancouver and Toronto. 
### - K-means is one of the best clustering methods so I am gonna apply it. 
### - Don't worry! Visualization will be provided to enhance the reading and understanding once the clusters are here, and the summary of these clusters from Van and Trt will be given.

## Data(Vancouver-based)

In [180]:
import pandas as pd

#Get the html source
page="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_V"
df = pd.read_html(page)[0]
df = df.iloc[:,4:7].reset_index(drop=True) # we'll choose the cities that are around the center of the region, which are Burnaby, Vancouver, North Vancouver, West Vancouver and Richmond.
print (df)

                                                    4  \
0   V5ABurnaby(Government Road / Lake City / SFU /...   
1     V5BBurnaby(Parkcrest-Aubrey / Ardingley-Sprott)   
2   V5CBurnaby(Burnaby Heights / Willingdon Height...   
3   V5EBurnaby(Lakeview-Mayfield / Richmond Park /...   
4          V5GBurnaby(Cascade-Schou / Douglas-Gilpin)   
5   V5HBurnaby(Maywood / Marlborough / Oakalla / W...   
6   V5JBurnaby(Suncrest / Sussex-Nelson / Clinton-...   
7                V5KVancouver(North Hastings-Sunrise)   
8              V5LVancouver(North Grandview-Woodland)   
9   V5MVancouver(South Hastings-Sunrise / North Re...   
10  V5NVancouver(South Grandview-Woodland / NE Ken...   
11  V5PVancouver(SE Kensington-Cedar Cottage / Vic...   
12            V5RVancouver(South Renfrew-Collingwood)   
13                            V5SVancouver(Killarney)   
14                  V5TVancouver(East Mount Pleasant)   
15  V5VVancouver(West Kensington-Cedar Cottage / N...   
16  V5WVancouver(SE Riley Park-

### Data wrangling
#### Extract the postcode

In [181]:
df5 = df[4].str[:3] 
df6 = df[5].str[:3]
df7 = df[6].str[:3]
dfpo=df5.append(df6).append(df7).reset_index(drop=True)
dfpo

0     V5A
1     V5B
2     V5C
3     V5E
4     V5G
5     V5H
6     V5J
7     V5K
8     V5L
9     V5M
10    V5N
11    V5P
12    V5R
13    V5S
14    V5T
15    V5V
16    V5W
17    V5X
18    V5Y
19    V5Z
20    V6A
21    V6B
22    V6C
23    V6E
24    V6G
25    V6H
26    V6J
27    V6K
28    V6L
29    V6M
30    V6N
31    V6P
32    V6R
33    V6S
34    V6T
35    V6V
36    V6W
37    V6X
38    V6Y
39    V6Z
40    V7A
41    V7B
42    V7C
43    V7E
44    V7G
45    V7H
46    V7J
47    V7K
48    V7L
49    V7M
50    V7N
51    V7P
52    V7R
53    V7S
54    V7T
55    V7V
56    V7W
57    V7X
58    V7Y
59    V7Z
dtype: object

#### Extract the borough

In [182]:
bo = df[4].str[3:]
bo
co = df[5].str[3:]
do = df[6].str[3:]
for i in range(0,20):
    bo[i] = bo[i].split('(')[0]
    co[i] = co[i].split('(')[0]
    do[i] = do[i].split('(')[0]


In [183]:
dfbo = bo.append(co).append(do).reset_index(drop=True)
dfbo

0                     Burnaby
1                     Burnaby
2                     Burnaby
3                     Burnaby
4                     Burnaby
5                     Burnaby
6                     Burnaby
7                   Vancouver
8                   Vancouver
9                   Vancouver
10                  Vancouver
11                  Vancouver
12                  Vancouver
13                  Vancouver
14                  Vancouver
15                  Vancouver
16                  Vancouver
17                  Vancouver
18                  Vancouver
19                  Vancouver
20                  Vancouver
21                  Vancouver
22                  Vancouver
23                  Vancouver
24                  Vancouver
25                  Vancouver
26                  Vancouver
27                  Vancouver
28                  Vancouver
29                  Vancouver
30                  Vancouver
31                  Vancouver
32                  Vancouver
33        

#### Extract the Neighborhood

In [184]:
n1 = df[4].str[3:]
n2 = df[5].str[3:]
n3 = df[6].str[3:]
for i in range(0,20):
    n1[i] = n1[i].split('(')[1].split(')')[0]
n1

0     Government Road / Lake City / SFU / Burnaby Mo...
1                   Parkcrest-Aubrey / Ardingley-Sprott
2     Burnaby Heights / Willingdon Heights / West Ce...
3     Lakeview-Mayfield / Richmond Park / Kingsway-B...
4                        Cascade-Schou / Douglas-Gilpin
5             Maywood / Marlborough / Oakalla / Windsor
6     Suncrest / Sussex-Nelson / Clinton-Glenwood / ...
7                                North Hastings-Sunrise
8                              North Grandview-Woodland
9     South Hastings-Sunrise / North Renfrew-Colling...
10    South Grandview-Woodland / NE Kensington-Cedar...
11    SE Kensington-Cedar Cottage / Victoria-Fraserview
12                            South Renfrew-Collingwood
13                                            Killarney
14                                  East Mount Pleasant
15    West Kensington-Cedar Cottage / NE Riley Park-...
16    SE Riley Park-Little Mountain / SW Kensington-...
17            SE Oakridge / East Marpole / South

In [185]:
n2
for i in range(0,15):
    n2[i] = n2[i].split('(')[1].split(')')[0]
n2[19] = 'SW Downtown'
n2[15] = 'Northeast'
n2[16] = 'Southeast'
n2[17] = 'North'
n2[18] = 'Central'

In [186]:
n2

0            Strathcona / Chinatown / Downtown Eastside
1     NE Downtown / Gastown / Harbour Centre / Inter...
2              Waterfront / Coal Harbour / Canada Place
3                           SE West End / Davie Village
4                            NW West End / Stanley Park
5     West Fairview / Granville Island / NE Shaughnessy
6           NW Shaughnessy / East Kitsilano / Quilchena
7                         Central Kitsilano / Greektown
8               NW Arbutus Ridge / NE Dunbar-Southlands
9     South Shaughnessy / NW Oakridge / NE Kerrisdal...
10    West Kerrisdale / South Dunbar-Southlands / Mu...
11           SE Kerrisdale / SW Oakridge / West Marpole
12           West Kitsilano / West Point Grey / Jericho
13    NW Dunbar-Southlands / Chaldecutt / South Univ...
14                                                  UBC
15                                            Northeast
16                                            Southeast
17                                              

In [187]:
n3
for i in range(4,13):
    n3[i] = n3[i].split(')')[1]
n3[1] = 'Sea Island / YVR'
n3[17] = 'Bentall Centre'
n3[18] = 'Pacific Centre'
n3[19] = 'Not assigned'
for j in range(13,17):
    n3[j] = n3[j][14:]
n3[0] = 'South'
n3[2] = 'Northwest'
n3[3] = 'Southwest'
n3

0                 South
1      Sea Island / YVR
2             Northwest
3             Southwest
4            Outer East
5            Inner East
6          East Central
7         North Central
8         South Central
9     Southwest Central
10    Northwest Central
11            Southwest
12            Northwest
13                North
14            Southeast
15                South
16                 West
17       Bentall Centre
18       Pacific Centre
19         Not assigned
Name: 6, dtype: object

In [188]:
neig = n1.append(n2).append(n3).reset_index(drop=True)
neig

0     Government Road / Lake City / SFU / Burnaby Mo...
1                   Parkcrest-Aubrey / Ardingley-Sprott
2     Burnaby Heights / Willingdon Heights / West Ce...
3     Lakeview-Mayfield / Richmond Park / Kingsway-B...
4                        Cascade-Schou / Douglas-Gilpin
5             Maywood / Marlborough / Oakalla / Windsor
6     Suncrest / Sussex-Nelson / Clinton-Glenwood / ...
7                                North Hastings-Sunrise
8                              North Grandview-Woodland
9     South Hastings-Sunrise / North Renfrew-Colling...
10    South Grandview-Woodland / NE Kensington-Cedar...
11    SE Kensington-Cedar Cottage / Victoria-Fraserview
12                            South Renfrew-Collingwood
13                                            Killarney
14                                  East Mount Pleasant
15    West Kensington-Cedar Cottage / NE Riley Park-...
16    SE Riley Park-Little Mountain / SW Kensington-...
17            SE Oakridge / East Marpole / South

#### Check the wrong format of data imported and adjust it 

In [189]:
dfbo[35] = dfbo[35][0:8]
dfbo[36] = dfbo[36][0:8]
dfbo[37] = dfbo[37][0:8]
dfbo[38] = dfbo[38][0:8]
dfbo[40] = dfbo[40][0:8]
dfbo[42] = dfbo[42][0:8]
dfbo[43] = dfbo[43][0:8]
dfbo[53] = dfbo[53][0:14]
dfbo[54] = dfbo[54][0:14]
dfbo[55] = dfbo[55][0:14]
dfbo[56] = dfbo[56][0:14]
dfbo

0              Burnaby
1              Burnaby
2              Burnaby
3              Burnaby
4              Burnaby
5              Burnaby
6              Burnaby
7            Vancouver
8            Vancouver
9            Vancouver
10           Vancouver
11           Vancouver
12           Vancouver
13           Vancouver
14           Vancouver
15           Vancouver
16           Vancouver
17           Vancouver
18           Vancouver
19           Vancouver
20           Vancouver
21           Vancouver
22           Vancouver
23           Vancouver
24           Vancouver
25           Vancouver
26           Vancouver
27           Vancouver
28           Vancouver
29           Vancouver
30           Vancouver
31           Vancouver
32           Vancouver
33           Vancouver
34           Vancouver
35            Richmond
36            Richmond
37            Richmond
38            Richmond
39           Vancouver
40            Richmond
41            Richmond
42            Richmond
43         

#### Organize the Vancouver data set

In [190]:
nd = pd.DataFrame(data={'Postcode':dfpo,'Borough':dfbo,'Neighborhood':neig})
nd

,Postcode,Borough,Neighborhood
0,V5A,Burnaby,Government Road / Lake City / SFU / Burnaby Mo...
1,V5B,Burnaby,Parkcrest-Aubrey / Ardingley-Sprott
2,V5C,Burnaby,Burnaby Heights / Willingdon Heights / West Ce...
3,V5E,Burnaby,Lakeview-Mayfield / Richmond Park / Kingsway-B...
4,V5G,Burnaby,Cascade-Schou / Douglas-Gilpin
5,V5H,Burnaby,Maywood / Marlborough / Oakalla / Windsor
6,V5J,Burnaby,Suncrest / Sussex-Nelson / Clinton-Glenwood / ...
7,V5K,Vancouver,North Hastings-Sunrise
8,V5L,Vancouver,North Grandview-Woodland
9,V5M,Vancouver,South Hastings-Sunrise / North Renfrew-Colling...


### Download the Canada Postal Data which includes the latitude and longitude.

In [191]:
vll = pd.read_csv("https://raw.githubusercontent.com/francistao1996/Coursera_Capstone/master/Canadian%20Postal%20Codes.csv")

### Get the center location based on each postal code

In [192]:
vll=vll.groupby(['Postcode'],as_index=False).mean()

### Merge the table 

In [193]:
nd =nd.merge(vll,on='Postcode')

In [194]:
nd

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,V5A,Burnaby,Government Road / Lake City / SFU / Burnaby Mo...,49.264485,-122.941439
1,V5B,Burnaby,Parkcrest-Aubrey / Ardingley-Sprott,49.273582,-122.977384
2,V5C,Burnaby,Burnaby Heights / Willingdon Heights / West Ce...,49.276181,-123.007376
3,V5E,Burnaby,Lakeview-Mayfield / Richmond Park / Kingsway-B...,49.226319,-122.957311
4,V5G,Burnaby,Cascade-Schou / Douglas-Gilpin,49.247746,-123.000456
5,V5H,Burnaby,Maywood / Marlborough / Oakalla / Windsor,49.230178,-122.998919
6,V5J,Burnaby,Suncrest / Sussex-Nelson / Clinton-Glenwood / ...,49.213681,-122.995491
7,V5K,Vancouver,North Hastings-Sunrise,49.279416,-123.041032
8,V5L,Vancouver,North Grandview-Woodland,49.278348,-123.065858
9,V5M,Vancouver,South Hastings-Sunrise / North Renfrew-Colling...,49.259952,-123.041817


## Data(Toronto-based)

In [195]:
import pandas as pd

#Get the html source
page1="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
tt = pd.read_html(page1)[0]

print (tt)

            0                 1  \
0    Postcode           Borough   
1         M1A      Not assigned   
2         M2A      Not assigned   
3         M3A        North York   
4         M4A        North York   
5         M5A  Downtown Toronto   
6         M5A  Downtown Toronto   
7         M6A        North York   
8         M6A        North York   
9         M7A      Queen's Park   
10        M8A      Not assigned   
11        M9A         Etobicoke   
12        M1B       Scarborough   
13        M1B       Scarborough   
14        M2B      Not assigned   
15        M3B        North York   
16        M4B         East York   
17        M4B         East York   
18        M5B  Downtown Toronto   
19        M5B  Downtown Toronto   
20        M6B        North York   
21        M7B      Not assigned   
22        M8B      Not assigned   
23        M9B         Etobicoke   
24        M9B         Etobicoke   
25        M9B         Etobicoke   
26        M9B         Etobicoke   
27        M9B       

### Drop the first row because it is column name and redefine the columns names

In [196]:
tt=tt.drop(0)
tt.columns = ['Postcode','Borough','Neighbourhood']
tt

,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Not assigned
10,M8A,Not assigned,Not assigned


### Ignore cells with a borough that is not assigned and reset the index

In [197]:
tt=tt[tt.Borough != "Not assigned"]

In [198]:
tt = tt.reset_index(drop=True)
tt

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


### Combine the rows

In [199]:
for i in range(0,211):
    for j in range(i+1,212):
        if tt.Postcode[i] == tt.Postcode[j]:
            tt.Neighbourhood[i] = tt.Neighbourhood[j]+ ',' + ' ' + tt.Neighbourhood[i]
tt

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,"Lawrence Manor, Lawrence Heights"
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,"Malvern, Rouge"
9,M1B,Scarborough,Malvern


### Drop the duplicate rows, keep the first which includes all neighbourhood names and reset the index

In [200]:
tt=tt.drop_duplicates(subset="Postcode",keep='first')
tt = tt.reset_index(drop=True)
tt

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Not assigned
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


### Revised the not assigned name

In [201]:
for i in range(0,len(tt)):
    if tt.Neighbourhood[i] == "Not assigned":
        tt.Neighbourhood[i] = tt.Borough[i]

### Use latitude and longitude to create a new toronto dataframe 

In [202]:
ll = pd.read_csv("https://cocl.us/Geospatial_data",header=0)
ll = ll.rename(columns={'Postal Code':'Postcode'})
tt=tt.merge(ll,on='Postcode')

In [203]:
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
%matplotlib inline
# import k-means from clustering stage
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.



In [204]:
toronto_data = tt[tt['Borough'].str.contains( 'Downtown Toronto|East Toronto|West Toronto|Central Toronto')].reset_index(drop=True)
toronto_data

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
5,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
6,M6G,Downtown Toronto,Christie,43.669542,-79.422564
7,M5H,Downtown Toronto,"Richmond, King, Adelaide",43.650571,-79.384568
8,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259
9,M5J,Downtown Toronto,"Union Station, Toronto Islands, Harbourfront East",43.640816,-79.381752


### Get the geographical coordinates of Toronto

In [205]:
addresst = 'Toronto, ON'

geolocator = Nominatim(user_agent="on_explorer")
locationt = geolocator.geocode(addresst)
latitudet = locationt.latitude
longitudet = locationt.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitudet, longitudet))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


### Get the geographical coordinates of Vancouver

In [206]:
addressv = 'Vancouver, BC'

geolocator = Nominatim(user_agent="on_explorer")
locationv = geolocator.geocode(addressv)
latitudev = locationv.latitude
longitudev = locationv.longitude
print('The geograpical coordinate of Vancouver are {}, {}.'.format(latitudev, longitudev))

The geograpical coordinate of Vancouver are 49.2608724, -123.1139529.


### Create a map of Toronto using latitude and longitude values

In [207]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitudet, longitudet], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'],toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Create a map of Vancouver using latitude and longitude values

In [208]:
# create map of Vancouver using latitude and longitude values
map_vancouver = folium.Map(location=[latitudev, longitudev], zoom_start=10)

# add markers to map
for lat, lng, label in zip(nd['Latitude'], nd['Longitude'],nd['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_vancouver)  
    
map_vancouver

In [209]:
CLIENT_ID = 'ECZVCJZZSENXGLEQXMMTUPBDHQADDRTMIGKNI11DSVVAUVQT' # your Foursquare ID
CLIENT_SECRET = 'TMCTBRNX4W3VNF1GSCO5LLLLCPXLLAH1FPLBUBVT51ITZ44E' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ECZVCJZZSENXGLEQXMMTUPBDHQADDRTMIGKNI11DSVVAUVQT
CLIENT_SECRET:TMCTBRNX4W3VNF1GSCO5LLLLCPXLLAH1FPLBUBVT51ITZ44E


In [210]:
import json
from pandas.io.json import json_normalize
import requests

In [211]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

## Let's explore the first neighborhood in Toronto.

In [212]:
toronto_data.loc[0, 'Neighbourhood']

'Regent Park, Harbourfront'

In [213]:
neighborhood_latitudet = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitudet =toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_namet = toronto_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_namet, 
                                                               neighborhood_latitudet, 
                                                               neighborhood_longitudet))

Latitude and longitude values of Regent Park, Harbourfront are 43.6542599, -79.3606359.


In [214]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitudet, 
    neighborhood_longitudet, 
    radius, 
    LIMIT)
url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=ECZVCJZZSENXGLEQXMMTUPBDHQADDRTMIGKNI11DSVVAUVQT&client_secret=TMCTBRNX4W3VNF1GSCO5LLLLCPXLLAH1FPLBUBVT51ITZ44E&v=20180605&ll=43.6542599,-79.3606359&radius=500&limit=100'

In [215]:
resultst = requests.get(url).json()
resultst

{'meta': {'code': 200, 'requestId': '5c847dabf594df76b6605ce7'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 49,
  'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486},
   'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label': 'display',
 

In [216]:
venuest = resultst['response']['groups'][0]['items']
    
nearby_venuest = json_normalize(venuest) # flatten JSON

# filter columns
filtered_columnst = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venuest =nearby_venuest.loc[:, filtered_columnst]

# filter the category for each row
nearby_venuest['venue.categories'] = nearby_venuest.apply(get_category_type, axis=1)

# clean columns
nearby_venuest.columns = [col.split(".")[-1] for col in nearby_venuest.columns]

nearby_venuest.head()

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Toronto Cooper Koo Family Cherry St YMCA Centre,Gym / Fitness Center,43.653191,-79.357947
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149


### Explore the neighborhood in Toronto (repeating)

In [217]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Create Toronto venues

In [219]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, King, Adelaide
Dufferin, Dovercourt Village
Union Station, Toronto Islands, Harbourfront East
Trinity, Little Portugal
Riverdale, The Danforth West
Toronto Dominion Centre, Design Exchange
Parkdale Village, Exhibition Place, Brockton
India Bazaar, The Beaches West
Victoria Hotel, Commerce Court
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill West, Forest Hill North
The Junction South, High Park
North Toronto West
Yorkville, North Midtown, The Annex
Roncesvalles, Parkdale
Davisville
University of Toronto, Harbord
Swansea, Runnymede
Summerhill East, Moore Park
Kensington Market, Grange Park, Chinatown
Summerhill West, South Hill, Rathnelly, Forest Hill SE, Deer Park
South Niagara, Railway Lands, King and Spadina, Harbourfront West, Island airport, Bathurst Quay, CN Tower
Rosedale
Stn A PO Boxes 25 The Esplanade
St. James Town, Cabbagetown
Und

### Analyzing each neighborhood

In [220]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columnst = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columnst]

toronto_onehot.head()

,Neighbourhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Group by neighbourhood and take the mean of the frequency of each category

In [221]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
1,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
2,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012195,...,0.000000,0.00,0.012195,0.000000,0.000000,0.012195,0.000000,0.000000,0.00,0.012195
3,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
4,Church and Wellesley,0.011905,0.011905,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011905,...,0.000000,0.00,0.000000,0.011905,0.011905,0.000000,0.011905,0.011905,0.00,0.011905
5,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
6,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
7,"Dufferin, Dovercourt Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
8,"Forest Hill West, Forest Hill North",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.250000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
9,"Garden District, Ryerson",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,...,0.000000,0.00,0.010000,0.000000,0.010000,0.010000,0.000000,0.000000,0.00,0.000000


### Print the each neighbourhood along with top 9 common venues and display

### Put that into a pandas dataframe

In [222]:
num_top_venues = 5
for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1        Cocktail Bar  0.05
2                 Pub  0.03
3  Seafood Restaurant  0.03
4         Cheese Shop  0.03


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0  Light Rail Station  0.12
1          Smoke Shop  0.06
2    Recording Studio  0.06
3             Brewery  0.06
4       Auto Workshop  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.17
1                Café  0.06
2  Italian Restaurant  0.05
3        Burger Joint  0.04
4     Bubble Tea Shop  0.04


----Christie----
               venue  freq
0      Grocery Store  0.19
1               Café  0.19
2               Park  0.12
3        Coffee Shop  0.06
4  Convenience Store  0.06


----Church and Wellesley----
                 venue  freq
0  Japanese Restaurant  0.07
1          Coffee Shop  0.07
2     Sushi Restaurant  0.05
3              Gay Bar  0.05
4           Restaurant  0.04



In [223]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [224]:
import numpy as np

In [225]:
num_top_venuest = 10

indicatorst = ['st', 'nd', 'rd']

# create columns according to number of top venues
columnst = ['Neighbourhood']
for ind in np.arange(num_top_venuest):
    try:
        columnst.append('{}{} Most Common Venue'.format(ind+1, indicatorst[ind]))
    except:
        columnst.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sortedt = pd.DataFrame(columns=columnst)
neighbourhoods_venues_sortedt['Neighbourhood'] =toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sortedt.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venuest)

neighbourhoods_venues_sortedt.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Cheese Shop,Pub,Café,Restaurant,Farmers Market,Seafood Restaurant,Steakhouse
1,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Farmers Market,Auto Workshop,Comic Shop,Pizza Place,Recording Studio,Restaurant,Burrito Place,Skate Park,Brewery
2,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Bubble Tea Shop,Bar,Burger Joint,Sandwich Place,Chinese Restaurant,Thai Restaurant,Spa
3,Christie,Café,Grocery Store,Park,Convenience Store,Nightclub,Diner,Baby Store,Italian Restaurant,Restaurant,Athletics & Sports
4,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Pub,Men's Store,Gastropub,Fast Food Restaurant,Mediterranean Restaurant


### Cluster neighborhoods in Toronto

In [226]:
# set number of clusters
tclusters = 6

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeanst = KMeans(n_clusters=tclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeanst.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 5, 0])

In [227]:
# add clustering labels
neighbourhoods_venues_sortedt.insert(0, 'Cluster Labels', kmeanst.labels_)

toronto_merged =toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sortedt.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head(10) # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Pub,Bakery,Park,Café,Breakfast Spot,Theater,Restaurant,Mexican Restaurant,Gym / Fitness Center
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Diner,Thai Restaurant,Ramen Restaurant,Bubble Tea Shop,Plaza,Japanese Restaurant
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Restaurant,Café,Hotel,Breakfast Spot,Park,Gastropub,Italian Restaurant,Cosmetics Shop,Clothing Store
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,3,Coffee Shop,Health Food Store,Pub,Other Great Outdoors,Neighborhood,Event Space,Falafel Restaurant,Farmers Market,Ethiopian Restaurant,Diner
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Cocktail Bar,Bakery,Cheese Shop,Pub,Café,Restaurant,Farmers Market,Seafood Restaurant,Steakhouse
5,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0,Coffee Shop,Café,Italian Restaurant,Bubble Tea Shop,Bar,Burger Joint,Sandwich Place,Chinese Restaurant,Thai Restaurant,Spa
6,M6G,Downtown Toronto,Christie,43.669542,-79.422564,0,Café,Grocery Store,Park,Convenience Store,Nightclub,Diner,Baby Store,Italian Restaurant,Restaurant,Athletics & Sports
7,M5H,Downtown Toronto,"Richmond, King, Adelaide",43.650571,-79.384568,0,Coffee Shop,Café,Bar,Steakhouse,Thai Restaurant,Hotel,Restaurant,Burger Joint,Bakery,American Restaurant
8,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259,0,Discount Store,Pharmacy,Supermarket,Bakery,Music Venue,Liquor Store,Brewery,Middle Eastern Restaurant,Café,Bar
9,M5J,Downtown Toronto,"Union Station, Toronto Islands, Harbourfront East",43.640816,-79.381752,0,Coffee Shop,Hotel,Aquarium,Café,Pizza Place,Italian Restaurant,Brewery,Restaurant,Scenic Lookout,History Museum


### Finally visualize it

In [228]:
# create map of Toronto
map_clusterst = folium.Map(location=[latitudet, longitudet], zoom_start=11)

# set color scheme for the clusters
x = np.arange(tclusters)
ys = [i + x + (i*x)**2 for i in range(tclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusterst)
       
map_clusterst

In [229]:
toronto_merged['Cluster Labels'].value_counts()


0    32
5     2
4     1
3     1
2     1
1     1
Name: Cluster Labels, dtype: int64

## Repeat for Vancouver

In [230]:
neighborhood_latitudev = nd.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitudev =nd.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_namev = nd.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_namev, 
                                                               neighborhood_latitudev, 
                                                               neighborhood_longitudev))

Latitude and longitude values of Government Road / Lake City / SFU / Burnaby Mountain are 49.26448458513447, -122.94143879139779.


In [231]:
urlv = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitudev, 
    neighborhood_longitudev, 
    radius, 
    LIMIT)
urlv # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=ECZVCJZZSENXGLEQXMMTUPBDHQADDRTMIGKNI11DSVVAUVQT&client_secret=TMCTBRNX4W3VNF1GSCO5LLLLCPXLLAH1FPLBUBVT51ITZ44E&v=20180605&ll=49.26448458513447,-122.94143879139779&radius=500&limit=100'

In [232]:
resultsv = requests.get(url).json()
resultsv

{'meta': {'code': 200, 'requestId': '5c847e0c4434b957cffe8a97'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 49,
  'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486},
   'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label': 'display',
 

In [233]:
venuesv = resultsv['response']['groups'][0]['items']
    
nearby_venuesv = json_normalize(venuesv) # flatten JSON

# filter columns
filtered_columnsv = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venuesv =nearby_venuesv.loc[:, filtered_columnsv]

# filter the category for each row
nearby_venuesv['venue.categories'] = nearby_venuesv.apply(get_category_type, axis=1)

# clean columns
nearby_venuesv.columns = [col.split(".")[-1] for col in nearby_venuesv.columns]

nearby_venuesv.head()

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Toronto Cooper Koo Family Cherry St YMCA Centre,Gym / Fitness Center,43.653191,-79.357947
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149


In [235]:
vancouver_venues = getNearbyVenues(names=nd['Neighborhood'],
                                   latitudes=nd['Latitude'],
                                   longitudes=nd['Longitude']
                                  )

Government Road / Lake City / SFU / Burnaby Mountain
Parkcrest-Aubrey / Ardingley-Sprott
Burnaby Heights / Willingdon Heights / West Central Valley
Lakeview-Mayfield / Richmond Park / Kingsway-Beresford
Cascade-Schou / Douglas-Gilpin
Maywood / Marlborough / Oakalla / Windsor
Suncrest / Sussex-Nelson / Clinton-Glenwood / West Big Bend
North Hastings-Sunrise
North Grandview-Woodland
South Hastings-Sunrise / North Renfrew-Collingwood
South Grandview-Woodland / NE Kensington-Cedar Cottage
SE Kensington-Cedar Cottage / Victoria-Fraserview
South Renfrew-Collingwood
Killarney
East Mount Pleasant
West Kensington-Cedar Cottage / NE Riley Park-Little Mountain
SE Riley Park-Little Mountain / SW Kensington-Cedar Cottage / NE Oakridge / North Sunset
SE Oakridge / East Marpole / South Sunset
West Mount Pleasant / West Riley Park-Little Mountain
East Fairview / South Cambie
Strathcona / Chinatown / Downtown Eastside
NE Downtown / Gastown / Harbour Centre / International Village / Victory Square / Yal

In [236]:
# one hot encoding
vancouver_onehot = pd.get_dummies(vancouver_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
vancouver_onehot['Neighbourhood'] = vancouver_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columnsv = [vancouver_onehot.columns[-1]] + list(vancouver_onehot.columns[:-1])
vancouver_onehot = vancouver_onehot[fixed_columnsv]

vancouver_onehot.head()

,Neighbourhood,ATM,African Restaurant,Airport,Airport Terminal,American Restaurant,Amphitheater,Arcade,Art Gallery,Arts & Crafts Store,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Water Park,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Government Road / Lake City / SFU / Burnaby Mo...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Government Road / Lake City / SFU / Burnaby Mo...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Government Road / Lake City / SFU / Burnaby Mo...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Parkcrest-Aubrey / Ardingley-Sprott,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Parkcrest-Aubrey / Ardingley-Sprott,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [237]:
vancouver_grouped = vancouver_onehot.groupby('Neighbourhood').mean().reset_index()
vancouver_grouped

,Neighbourhood,ATM,African Restaurant,Airport,Airport Terminal,American Restaurant,Amphitheater,Arcade,Art Gallery,Arts & Crafts Store,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Water Park,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Bentall Centre,0.00,0.000000,0.000000,0.000000,0.013514,0.000000,0.000000,0.013514,0.000000,...,0.000000,0.000000,0.0000,0.013514,0.013514,0.000000,0.013514,0.000000,0.00000,0.040541
1,Burnaby Heights / Willingdon Heights / West Ce...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
2,Cascade-Schou / Douglas-Gilpin,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
3,Central,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
4,Central Kitsilano / Greektown,0.00,0.000000,0.000000,0.000000,0.024390,0.000000,0.000000,0.000000,0.000000,...,0.024390,0.000000,0.0000,0.024390,0.000000,0.000000,0.000000,0.024390,0.02439,0.024390
5,East Central,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
6,East Fairview / South Cambie,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.040000,0.000000,0.0000,0.040000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
7,East Mount Pleasant,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.044444,...,0.000000,0.000000,0.0000,0.044444,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
8,Government Road / Lake City / SFU / Burnaby Mo...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
9,Inner East,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000


In [238]:
num_top_venuesv = 5
for hood in vancouver_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = vancouver_grouped[vancouver_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venuesv))
    print('\n')

----Bentall Centre----
                venue  freq
0               Hotel  0.11
1          Restaurant  0.07
2  Italian Restaurant  0.05
3              Bakery  0.04
4  Seafood Restaurant  0.04


----Burnaby Heights / Willingdon Heights / West Central Valley----
                venue  freq
0      Shopping Plaza   1.0
1                 ATM   0.0
2        Noodle House   0.0
3  Miscellaneous Shop   0.0
4   Mobile Phone Shop   0.0


----Cascade-Schou / Douglas-Gilpin----
         venue  freq
0  Snack Place  0.22
1  Coffee Shop  0.11
2  Bus Station  0.11
3    Bookstore  0.11
4     Bus Stop  0.11


----Central----
                venue  freq
0   Convenience Store  0.29
1    Sushi Restaurant  0.14
2            Pharmacy  0.14
3      Clothing Store  0.14
4  Chinese Restaurant  0.14


----Central Kitsilano / Greektown----
                 venue  freq
0          Coffee Shop  0.15
1        Grocery Store  0.07
2          Pizza Place  0.05
3  Japanese Restaurant  0.05
4           Restaurant  0.05


---

            venue  freq
0    Liquor Store  0.33
1  Sandwich Place  0.33
2   Grocery Store  0.33
3             ATM  0.00
4    Noodle House  0.00


----UBC----
                           venue  freq
0                           Café  0.15
1                         Garden  0.15
2              French Restaurant  0.08
3  Vegetarian / Vegan Restaurant  0.08
4                            Pub  0.08


----Waterfront / Coal Harbour / Canada Place----
          venue  freq
0         Hotel  0.09
1   Coffee Shop  0.07
2          Café  0.05
3    Steakhouse  0.04
4  Dessert Shop  0.03


----West----
                       venue  freq
0                 Playground  0.33
1           Tapas Restaurant  0.33
2            Harbor / Marina  0.33
3               Noodle House  0.00
4  Middle Eastern Restaurant  0.00


----West Fairview / Granville Island / NE Shaughnessy----
            venue  freq
0  Breakfast Spot  0.22
1       Pet Store  0.11
2     Bus Station  0.11
3        Bus Stop  0.11
4            Café  0

In [239]:
num_top_venuesvv = 10

indicatorsv = ['st', 'nd', 'rd']

# create columns according to number of top venues
columnsv = ['Neighbourhood']
for ind in np.arange(num_top_venuesvv):
    try:
        columnsv.append('{}{} Most Common Venue'.format(ind+1, indicatorsv[ind]))
    except:
        columnsv.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sortedv = pd.DataFrame(columns=columnsv)
neighbourhoods_venues_sortedv['Neighbourhood'] =vancouver_grouped['Neighbourhood']

for ind in np.arange(vancouver_grouped.shape[0]):
    neighbourhoods_venues_sortedv.iloc[ind, 1:] = return_most_common_venues(vancouver_grouped.iloc[ind, :], num_top_venuesvv)

neighbourhoods_venues_sortedv.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bentall Centre,Hotel,Restaurant,Italian Restaurant,Seafood Restaurant,Bakery,Breakfast Spot,Coffee Shop,Concert Hall,Yoga Studio,Burger Joint
1,Burnaby Heights / Willingdon Heights / West Ce...,Shopping Plaza,Yoga Studio,Comfort Food Restaurant,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Fair
2,Cascade-Schou / Douglas-Gilpin,Snack Place,Bookstore,Auto Garage,Bus Station,Bus Stop,Sandwich Place,Park,Coffee Shop,Department Store,Dessert Shop
3,Central,Convenience Store,Pharmacy,Clothing Store,Sushi Restaurant,Chinese Restaurant,Sandwich Place,Donut Shop,Fast Food Restaurant,Farmers Market,Farm
4,Central Kitsilano / Greektown,Coffee Shop,Grocery Store,Restaurant,Japanese Restaurant,Pharmacy,Pizza Place,Fast Food Restaurant,Supplement Shop,Food Truck,Breakfast Spot


In [240]:
# set number of clusters
vclusters = 6

vancouver_grouped_clustering = vancouver_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeansv = KMeans(n_clusters=vclusters, random_state=0).fit(vancouver_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeansv.labels_[0:10] 

array([0, 1, 0, 0, 0, 5, 0, 0, 3, 0])

In [241]:
# add clustering labels
neighbourhoods_venues_sortedv.insert(0, 'Cluster Labels', kmeansv.labels_)

vancouver_merged =nd

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
vancouver_merged = vancouver_merged.join(neighbourhoods_venues_sortedv.set_index('Neighbourhood'), on='Neighborhood').dropna()

vancouver_merged.head(10) # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,V5A,Burnaby,Government Road / Lake City / SFU / Burnaby Mo...,49.264485,-122.941439,3.0,Burger Joint,Golf Course,Golf Driving Range,Yoga Studio,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
1,V5B,Burnaby,Parkcrest-Aubrey / Ardingley-Sprott,49.273582,-122.977384,0.0,Stadium,Bus Stop,Playground,Martial Arts Dojo,Golf Course,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market,Farm
2,V5C,Burnaby,Burnaby Heights / Willingdon Heights / West Ce...,49.276181,-123.007376,1.0,Shopping Plaza,Yoga Studio,Comfort Food Restaurant,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Fair
3,V5E,Burnaby,Lakeview-Mayfield / Richmond Park / Kingsway-B...,49.226319,-122.957311,0.0,Mobile Phone Shop,Bus Stop,Yoga Studio,Concert Hall,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
4,V5G,Burnaby,Cascade-Schou / Douglas-Gilpin,49.247746,-123.000456,0.0,Snack Place,Bookstore,Auto Garage,Bus Station,Bus Stop,Sandwich Place,Park,Coffee Shop,Department Store,Dessert Shop
5,V5H,Burnaby,Maywood / Marlborough / Oakalla / Windsor,49.230178,-122.998919,0.0,Clothing Store,Coffee Shop,Cosmetics Shop,Chinese Restaurant,Café,Toy / Game Store,Sporting Goods Shop,Lingerie Store,Tea Room,Pharmacy
6,V5J,Burnaby,Suncrest / Sussex-Nelson / Clinton-Glenwood / ...,49.213681,-122.995491,0.0,Liquor Store,Sandwich Place,Grocery Store,Yoga Studio,Donut Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
7,V5K,Vancouver,North Hastings-Sunrise,49.279416,-123.041032,0.0,Theme Park Ride / Attraction,Vietnamese Restaurant,Event Space,Theme Park,Office,Bridal Shop,Bus Line,Bus Station,Sandwich Place,Fair
8,V5L,Vancouver,North Grandview-Woodland,49.278348,-123.065858,0.0,Pizza Place,Coffee Shop,Asian Restaurant,Grocery Store,Theater,Fried Chicken Joint,Italian Restaurant,Bakery,Japanese Restaurant,Sushi Restaurant
9,V5M,Vancouver,South Hastings-Sunrise / North Renfrew-Colling...,49.259952,-123.041817,0.0,Coffee Shop,Pizza Place,Arts & Crafts Store,Bus Stop,Sandwich Place,Restaurant,Liquor Store,Chinese Restaurant,Grocery Store,Office


In [242]:
# create map of Toronto
map_clustersv = folium.Map(location=[latitudev, longitudev], zoom_start=10)

# set color scheme for the clusters
x = np.arange(vclusters)
ys = [i + x + (i*x)**2 for i in range(vclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(vancouver_merged['Latitude'], vancouver_merged['Longitude'], vancouver_merged['Neighborhood'], vancouver_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clustersv)
       
map_clustersv

In [243]:
vancouver_merged.groupby(['Cluster Labels']).head(20)

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,V5A,Burnaby,Government Road / Lake City / SFU / Burnaby Mo...,49.264485,-122.941439,3.0,Burger Joint,Golf Course,Golf Driving Range,Yoga Studio,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
1,V5B,Burnaby,Parkcrest-Aubrey / Ardingley-Sprott,49.273582,-122.977384,0.0,Stadium,Bus Stop,Playground,Martial Arts Dojo,Golf Course,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market,Farm
2,V5C,Burnaby,Burnaby Heights / Willingdon Heights / West Ce...,49.276181,-123.007376,1.0,Shopping Plaza,Yoga Studio,Comfort Food Restaurant,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Fair
3,V5E,Burnaby,Lakeview-Mayfield / Richmond Park / Kingsway-B...,49.226319,-122.957311,0.0,Mobile Phone Shop,Bus Stop,Yoga Studio,Concert Hall,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
4,V5G,Burnaby,Cascade-Schou / Douglas-Gilpin,49.247746,-123.000456,0.0,Snack Place,Bookstore,Auto Garage,Bus Station,Bus Stop,Sandwich Place,Park,Coffee Shop,Department Store,Dessert Shop
5,V5H,Burnaby,Maywood / Marlborough / Oakalla / Windsor,49.230178,-122.998919,0.0,Clothing Store,Coffee Shop,Cosmetics Shop,Chinese Restaurant,Café,Toy / Game Store,Sporting Goods Shop,Lingerie Store,Tea Room,Pharmacy
6,V5J,Burnaby,Suncrest / Sussex-Nelson / Clinton-Glenwood / ...,49.213681,-122.995491,0.0,Liquor Store,Sandwich Place,Grocery Store,Yoga Studio,Donut Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
7,V5K,Vancouver,North Hastings-Sunrise,49.279416,-123.041032,0.0,Theme Park Ride / Attraction,Vietnamese Restaurant,Event Space,Theme Park,Office,Bridal Shop,Bus Line,Bus Station,Sandwich Place,Fair
8,V5L,Vancouver,North Grandview-Woodland,49.278348,-123.065858,0.0,Pizza Place,Coffee Shop,Asian Restaurant,Grocery Store,Theater,Fried Chicken Joint,Italian Restaurant,Bakery,Japanese Restaurant,Sushi Restaurant
9,V5M,Vancouver,South Hastings-Sunrise / North Renfrew-Colling...,49.259952,-123.041817,0.0,Coffee Shop,Pizza Place,Arts & Crafts Store,Bus Stop,Sandwich Place,Restaurant,Liquor Store,Chinese Restaurant,Grocery Store,Office


In [244]:
vancouver_merged['Cluster Labels'].value_counts()

0.0    43
5.0     8
4.0     2
2.0     1
1.0     1
3.0     1
Name: Cluster Labels, dtype: int64

### Examine the clusters

### Toronto cluster 1:

In [245]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Pub,Bakery,Park,Café,Breakfast Spot,Theater,Restaurant,Mexican Restaurant,Gym / Fitness Center
1,Downtown Toronto,0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Diner,Thai Restaurant,Ramen Restaurant,Bubble Tea Shop,Plaza,Japanese Restaurant
2,Downtown Toronto,0,Coffee Shop,Restaurant,Café,Hotel,Breakfast Spot,Park,Gastropub,Italian Restaurant,Cosmetics Shop,Clothing Store
4,Downtown Toronto,0,Coffee Shop,Cocktail Bar,Bakery,Cheese Shop,Pub,Café,Restaurant,Farmers Market,Seafood Restaurant,Steakhouse
5,Downtown Toronto,0,Coffee Shop,Café,Italian Restaurant,Bubble Tea Shop,Bar,Burger Joint,Sandwich Place,Chinese Restaurant,Thai Restaurant,Spa
6,Downtown Toronto,0,Café,Grocery Store,Park,Convenience Store,Nightclub,Diner,Baby Store,Italian Restaurant,Restaurant,Athletics & Sports
7,Downtown Toronto,0,Coffee Shop,Café,Bar,Steakhouse,Thai Restaurant,Hotel,Restaurant,Burger Joint,Bakery,American Restaurant
8,West Toronto,0,Discount Store,Pharmacy,Supermarket,Bakery,Music Venue,Liquor Store,Brewery,Middle Eastern Restaurant,Café,Bar
9,Downtown Toronto,0,Coffee Shop,Hotel,Aquarium,Café,Pizza Place,Italian Restaurant,Brewery,Restaurant,Scenic Lookout,History Museum
10,West Toronto,0,Bar,Coffee Shop,Asian Restaurant,Restaurant,Vietnamese Restaurant,Café,Bakery,Pizza Place,Cocktail Bar,Men's Store


### cluster 2

In [257]:
ct2 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

### cluster 3

In [258]:
ct3 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

### cluster 4

In [259]:
ct4 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

### cluster 5

In [260]:
ct5 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

### cluster 6

In [261]:
ct6 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

### Combine the clusters apart from 1

In [262]:
ctc = pd.concat([ct2,ct3,ct4,ct5,ct6])

In [263]:
ctc

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Central Toronto,1,Park,Swim School,Bus Line,Yoga Studio,Discount Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
18,Central Toronto,2,Garden,Pool,Yoga Studio,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
3,East Toronto,3,Coffee Shop,Health Food Store,Pub,Other Great Outdoors,Neighborhood,Event Space,Falafel Restaurant,Farmers Market,Ethiopian Restaurant,Diner
28,Central Toronto,4,Playground,Diner,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
20,Central Toronto,5,Park,Trail,Sushi Restaurant,Jewelry Store,Yoga Studio,Discount Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
32,Downtown Toronto,5,Park,Playground,Trail,Diner,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


### Vancouver cluster 1

In [251]:
vancouver_merged.loc[vancouver_merged['Cluster Labels'] == 0, vancouver_merged.columns[[1] + list(range(5, vancouver_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Burnaby,0.0,Stadium,Bus Stop,Playground,Martial Arts Dojo,Golf Course,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market,Farm
3,Burnaby,0.0,Mobile Phone Shop,Bus Stop,Yoga Studio,Concert Hall,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
4,Burnaby,0.0,Snack Place,Bookstore,Auto Garage,Bus Station,Bus Stop,Sandwich Place,Park,Coffee Shop,Department Store,Dessert Shop
5,Burnaby,0.0,Clothing Store,Coffee Shop,Cosmetics Shop,Chinese Restaurant,Café,Toy / Game Store,Sporting Goods Shop,Lingerie Store,Tea Room,Pharmacy
6,Burnaby,0.0,Liquor Store,Sandwich Place,Grocery Store,Yoga Studio,Donut Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
7,Vancouver,0.0,Theme Park Ride / Attraction,Vietnamese Restaurant,Event Space,Theme Park,Office,Bridal Shop,Bus Line,Bus Station,Sandwich Place,Fair
8,Vancouver,0.0,Pizza Place,Coffee Shop,Asian Restaurant,Grocery Store,Theater,Fried Chicken Joint,Italian Restaurant,Bakery,Japanese Restaurant,Sushi Restaurant
9,Vancouver,0.0,Coffee Shop,Pizza Place,Arts & Crafts Store,Bus Stop,Sandwich Place,Restaurant,Liquor Store,Chinese Restaurant,Grocery Store,Office
10,Vancouver,0.0,Café,Bookstore,Church,Farmers Market,Park,Bike Shop,Sushi Restaurant,Beach,Lake,Dog Run
11,Vancouver,0.0,Japanese Restaurant,Pizza Place,Convenience Store,Asian Restaurant,Fried Chicken Joint,Bus Stop,Fish Market,Bus Station,Thrift / Vintage Store,Noodle House


### cluster 2

In [264]:
cv2=vancouver_merged.loc[vancouver_merged['Cluster Labels'] == 1, vancouver_merged.columns[[1] + list(range(5, vancouver_merged.shape[1]))]]

### cluster 3

In [265]:
cv3=vancouver_merged.loc[vancouver_merged['Cluster Labels'] == 2, vancouver_merged.columns[[1] + list(range(5, vancouver_merged.shape[1]))]]

### cluster 4

In [266]:
cv4=vancouver_merged.loc[vancouver_merged['Cluster Labels'] == 3, vancouver_merged.columns[[1] + list(range(5, vancouver_merged.shape[1]))]]

### cluster 5

In [267]:
cv5=vancouver_merged.loc[vancouver_merged['Cluster Labels'] == 4, vancouver_merged.columns[[1] + list(range(5, vancouver_merged.shape[1]))]]

### cluster 6

In [268]:
cv6=vancouver_merged.loc[vancouver_merged['Cluster Labels'] == 5, vancouver_merged.columns[[1] + list(range(5, vancouver_merged.shape[1]))]]

### combine 2-6

In [269]:
cvc = pd.concat([cv2,cv3,cv4,cv5,cv6])
cvc

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Burnaby,1.0,Shopping Plaza,Yoga Studio,Comfort Food Restaurant,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Fair
32,Vancouver,2.0,Gym / Fitness Center,Yoga Studio,Donut Shop,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Fair
0,Burnaby,3.0,Burger Joint,Golf Course,Golf Driving Range,Yoga Studio,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
48,North Vancouver,4.0,Playground,Café,Convenience Store,Cosmetics Shop,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
56,West Vancouver,4.0,Playground,Tapas Restaurant,Harbor / Marina,Dog Run,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Fair
17,Vancouver,5.0,Indian Restaurant,Park,Yoga Studio,Donut Shop,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
42,Richmond,5.0,Eastern European Restaurant,Park,Sushi Restaurant,Farm,Basketball Court,Trail,Yoga Studio,Filipino Restaurant,Field,Fast Food Restaurant
44,North Vancouver,5.0,Dog Run,Pub,Park,Soccer Field,Yoga Studio,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Fair
46,North Vancouver,5.0,Distribution Center,Park,Trail,Dog Run,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Fair
47,North Vancouver,5.0,Convenience Store,Dance Studio,Park,Eastern European Restaurant,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
